# EDA Of Adult Asthma-COPD Audit Data 
https://www.data.gov.uk/dataset/84cd03c7-5ba5-47f1-9aed-f43d811348d2/national-asthma-and-copd-audit-programme-nacap-adult-asthma-and-copd-2021-organisational-audit
> The `asthmadata.csv` file contains data for 159 hospitals with statistics related to respiratory illnesses (asthma and COPD, Chronic Obstructive Pulmonary Disease) such as admissions to hospitals, bed space, staff availibility etc.
<br><br>
> Here we use pandas to handle the csv data file and load the data into a dataframe. Additional wrangling is required to establish well formated multiple headers for the dataframe

In [55]:
# imports for data handling and displaying
%matplotlib inline
import numpy as np
import pandas as pd
import json
# to enable interactivity
import ipywidgets as widgets
# plotting packages
import plotly.express as px
import plotly.graph_objects as go
import folium

# import for notebook interactivity
from ipywidgets import interact, interact_manual, interactive, Layout
from IPython.display import display, Markdown

# to print all output rather than the default last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # other options  'all', 'none', 'last' and 'last_expr

# change the max rows displayed
pd.options.display.max_rows = 20

<br><br>
## Function Definitions
> Here functions used throughout this notebook are defined

In [56]:
def list_level_zero_headers(df):
    # returns level zero headers
    return df.columns.get_level_values(0).unique().tolist()

def list_level_one_headers(df):
    # returns level one headers
    sub_cats_list=[]
    for category in list_level_zero_headers(df):
        for subcat in df[category].columns.tolist():
            sub_cats_list.append(subcat)
        
    return sub_cats_list

def convert_column_indexes_to_dict(df):
    # returns dictionary of subcategories mapped to catergories 
    
    # each level 1 header is set as a key, and their corresponding level 2 header as a value
    # this allows for a dictionary to be returned showing the category each subcategory sits in
    categories_tup_list=df.columns.tolist()
    categories_dict=dict(map(reversed, categories_tup_list))
    
    return categories_dict

def find_category(category_dict, column_name):
    # return category of data based on column name given 
    
    return category_dict[column_name]

def get_specific_column_from_multiindex(df, column_name):
    # return specific column in multiindex given name of column
    
    i=[index for index, tup in enumerate(df.columns.tolist()) if tup[1]==column_name]
    
    rows = len(df.index)
    
    return df.iloc[0:rows,i]

def show_data_by_category(df, num_rows=2, num_cols=10):
    # returns interactive display of all data in df, with category selection 
    
    categories_list=list_level_zero_headers(df)
    @interact
    def display(
        category=widgets.Dropdown(options=categories_list, 
                                  description="Category of Data",
                                  style={'description_width': '40%'}),
        rows=widgets.IntSlider(min=1,max=len(df.index),step=1,value=num_rows)):
            pd.set_option('display.max_columns', None, 'display.max_rows', None)
            
            sub_cats_list=df[category].columns.tolist()

            return df[category].iloc[0:rows,0:len(df[category])]

def show_data_with_row_control(df, num_rows=2):
    # returns interactive display of df with num row view control only
    
    print("\n\nScroll to view data...\n")
    @interact
    def show_number_of_rows(
        rows=widgets.IntSlider(min=1,max=len(df.index),step=1,value=num_rows)):
            pd.set_option('display.max_columns', None, 'display.max_rows', None)

            return df.iloc[0:rows,0:len(df)]
        
def show_data_with_row_column_control(df, num_rows=2, num_cols=10):
    # returns interactive display of df with num row and column control
    
    print("\n\nScroll to view data...\n")
    @interact
    def show_number_of_rows(
        rows=widgets.IntSlider(min=1,max=len(df.index),step=1,value=num_rows),
        columns=widgets.IntSlider(min=1,max=len(df),step=1,value=num_cols)):
            pd.set_option('display.max_columns', None, 'display.max_rows', None)

            return df.iloc[0:rows,0:columns]

def show_specific_columns_of_data(df, columns_to_display, num_rows=2):
    # returns interactive display of specific columns, given as a list of names
    
    print("\n\nScroll to view data...\n")
    @interact
    def display(
        rows=widgets.IntSlider(min=1,max=len(df.index),step=1,value=num_rows)):
            pd.set_option('display.max_columns', None, 'display.max_rows', None)
            
            i=[index for index, tup in enumerate(df.columns.tolist()) if tup[1] in columns_to_display]
            
            return df.iloc[0:rows,i]
    

<br><br><br><br><br><br><br><br>
## Simple Reading Of Data
> Here we read the data without the column groupings in the csv file
<br>
<table align='centre'>
  <tr>
    <th scope="col" style="text-align:left"></th>
    <th scope="col" style="text-align:left">Col A</th>
    <th scope="col" style="text-align:left">Col B</th>
    <th scope="col" style="text-align:left">Col C</th>
    <th scope="col" style="text-align:left">Col D</th>
  </tr>
  <tr>
    <th scope="row">Row 1</th>
    <td>data</td>
    <td>data</td>
    <td>data</td>
    <td>data</td>
  </tr>
  <tr>
    <th scope="row">Row 2</th>
    <td>data</td>
    <td>data</td>
    <td>data</td>
    <td>data</td>
  </tr>
</table>
<br>

In [57]:
# read data with pandas, skipping row with dataset title and grouping titles
# ideally we'd want to read data including the groupings, perhaps as metadata of some sort for columns 

datacsv = "asthmadata.csv"


df_simple = pd.read_csv(datacsv, skiprows=2, encoding='cp1252')

<br><br>
> We see that the columns are of type Index

In [58]:
print(type(df_simple.columns))

<class 'pandas.core.indexes.base.Index'>


<br><br>
## Interactive View of Simple Dataframe
> Set the number of rows and columns displayed
<br>
> Default rows: 2
<br>
> Default columns: 10

In [59]:
show_data_with_row_column_control(df_simple)

,Region,Integrated Care System (ICS),Trust/Local Health Board name,Hospital name (d = 159),Number of adult medical emergency admissions in 2020/21 financial year,Number of adult medical emergency admissions in 2020/21 financial year per medical bed,Number of respiratory coded medical emergency admissions in 2020/21 financial year,Number of respiratory coded emergency admissions in 2020/21 financial year per respiratory bed,Number of COPD coded emergency admissions in 2020/21 financial year,"Number of COPD coded emergency admissions in 2020/21 financial year per 1,000 adult medical emergency admissions"
0,East of England,"Bedfordshire, Luton and Milton Keynes",Luton and Dunstable University Hospital NHS Fo...,Luton & Dunstable Hospital,39628,104.0,5265,160,806,20
1,East of England,"Bedfordshire, Luton and Milton Keynes",Milton Keynes University Hospital NHS Foundati...,Milton Keynes General Hospital,13639,43.0,2482,43,346,25


interactive(children=(IntSlider(value=2, description='rows', max=159, min=1), IntSlider(value=10, description=…

<br><br>
## Understanding Data Shape
> This is just for easy reading of data variables available to analyse

In [60]:
# show dataset info

df_simple.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Columns: 157 entries, Region to No formal transition plan arrangements
dtypes: float64(25), int64(7), object(125)
memory usage: 195.2+ KB


> The dataset is 159 rows by 159 columns, with 25 float (float64) columns, 7 integer (int64) columns and 125 object columns - the object columns are likely to contain mixed datatype values, and will need cleaning before plotting

<br><br><br><br><br><br><br><br>
## Reading The Data With Multiple Header Rows
> Here we enable **hierarchical indexing** (MultiIndex)
<br>
> To read more about hierarchical indexing: https://pandas.pydata.org/docs/user_guide/advanced.html
<br><br>
> The dataset contains groups for columns, which provide useful information which we wish to preserve
<br><br> Our dataframe is reformatted as shown below:

<table align='centre'>
  <col>
  <colgroup span="2"></colgroup>
  <colgroup span="2"></colgroup>
  <tr>
    <td rowspan="2"></td>
    <th colspan="2" scope="colgroup" style="text-align:left">Cat A</th>
    <th colspan="2" scope="colgroup" style="text-align:left">Cat B</th>
  </tr>
  <tr>
    <th scope="col" style="text-align:left">Col A</th>
    <th scope="col" style="text-align:left">Col B</th>
    <th scope="col" style="text-align:left">Col C</th>
    <th scope="col" style="text-align:left">Col D</th>
  </tr>
  <tr>
    <th scope="row">Row 1</th>
    <td>data</td>
    <td>data</td>
    <td>data</td>
    <td>data</td>
  </tr>
  <tr>
    <th scope="row">Row 2</th>
    <td>data</td>
    <td>data</td>
    <td>data</td>
    <td>data</td>
  </tr>
</table>
<br>

In [61]:
datacsv = "asthmadata.csv"

df = pd.read_csv(datacsv, skiprows=1,  header=[0,1], encoding='cp1252')

# ensuring no unnamed columns exist
last_non_unnamed=''
for column in df.columns:
    if 'Unnamed' not in column[0]:
        last_non_unnamed=column[0]
        
    df.rename(columns={column[0]:last_non_unnamed}, inplace=True)
    
# view first two rows of data ONLY 
df.head(2)


Location                                          \
             Region            Integrated Care System (ICS)   
0  East of England   Bedfordshire, Luton and Milton Keynes    
1  East of England   Bedfordshire, Luton and Milton Keynes    

                                                      \
                       Trust/Local Health Board name   
0  Luton and Dunstable University Hospital NHS Fo...   
1  Milton Keynes University Hospital NHS Foundati...   

                                   \
          Hospital name (d = 159)   
0      Luton & Dunstable Hospital   
1  Milton Keynes General Hospital   

                                                        Admission numbers  \
  Number of adult medical emergency admissions in 2020/21 financial year    
0                                              39628                        
1                                              13639                        

                                                                                          \
  Number of adult medical emergency admissions in 2020/21 financial year per medical bed   
0                                              104.0                                       
1                                               43.0                                       

                                                                                       \
  Number of respiratory coded medical emergency admissions in 2020/21 financial year    
0                                               5265                                    
1                                               2482                                    

                                                                                                  \
  Number of respiratory coded emergency admissions in 2020/21 financial year per respiratory bed   
0                                                160                                               
1                                                 43                                               

                                                                        \
  Number of COPD coded emergency admissions in 2020/21 financial year    
0                                                806                     
1                                                346                     

                                                                                                                     \
  Number of COPD coded emergency admissions in 2020/21 financial year per 1,000 adult medical emergency admissions    
0                                                 20                                                                  
1                                                 25                                                                  

                                                                                                                                                     \
  Total number of COPD coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year   
0                                                316                                                                                                  
1                                                123                                                                                                  

                                                                                                                                                                    \
  National average proportion of COPD coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year   
0                                                 39                                                                                                                 
1                                                 36                   

> Above we see our DataSet as a MultiIndex with two header levels - throughout the rest of this notebook, these are referred to as *categories* and *subcategories*

In [62]:
print(type(df.columns))

<class 'pandas.core.indexes.multi.MultiIndex'>



> We now see that dataframe.columns now returns a MultiIndex object

<br><br>
## Data Groups
> Now that we have enabled MultiIndex, we can begin to analyse groups of data in the dataframe
<br><br>
Note an alternative way to access the MultiIndex columns:
<br>`mi = pd.MultiIndex.from_frame(df)`
<br><br>
>Specific sets of columns of data can be accessed as a series by running the following commands:
<br>
>> *eg. for all Location columns:*
<br>
`df["Location"]`
<br><br>
*eg. for Region column under Location:*
<br>
`df["Location", "Region"]`

### View All Subcategory Data
> Here we can view the tabular representation of all subcategories of data within each category

In [63]:
categories_list=list_level_zero_headers(df)

print("\nHere are all the categories of data variables that exist within the dataset")
# for header in categories_list:
#     print(header)

show_data_by_category(df, 5)

,Region,Integrated Care System (ICS),Trust/Local Health Board name,Hospital name (d = 159)
0,East of England,"Bedfordshire, Luton and Milton Keynes",Luton and Dunstable University Hospital NHS Fo...,Luton & Dunstable Hospital
1,East of England,"Bedfordshire, Luton and Milton Keynes",Milton Keynes University Hospital NHS Foundati...,Milton Keynes General Hospital
2,East of England,Cambridgeshire and Peterborough,Cambridge University Hospitals NHS Foundation ...,Addenbrooke's Hospital
3,East of England,Cambridgeshire and Peterborough,North West Anglia NHS Foundation Trust,Hinchingbrooke Hospital
4,East of England,Cambridgeshire and Peterborough,North West Anglia NHS Foundation Trust,Peterborough City Hospital


interactive(children=(Dropdown(description='Category of Data', options=('Location', 'Admission numbers', 'Beds…

<br><br><br><br><br><br><br><br>
## Cleaning the Data | Dropping Columns
> Upon analysis of all groups of data, entire categories are removed, and some subcategories from remaining categories are also removed
### Tool to make copying and pasting column names easier
> Note: this tool is useful for manual deletion of columns or any other requirement for string column names

In [64]:
# to view markdown for all of the subcategories within a category: 
def show_cats_and_subcats(df):
    @interact
    def show_sub_categories(category=widgets.Dropdown(options=categories_list, 
                            description="Category of Data",
                            style={'description_width': '40%'})
                           ):
        sub_cats_list=df[category].columns.tolist()
        prefix='<b>Category:</b><br><li>"{}"<br><b>Subcategories:</b>'.format(category)
        sub_cats_list_joined=prefix+''.join('<br><li>"{}"'.format(word) for word in map(str, sub_cats_list))

        return display(Markdown(sub_cats_list_joined))
    
show_cats_and_subcats(df)

<b>Category:</b><br><li>"Location"<br><b>Subcategories:</b><br><li>"Region"<br><li>"Integrated Care System (ICS)"<br><li>"Trust/Local Health Board name"<br><li>"Hospital name (d = 159)"

interactive(children=(Dropdown(description='Category of Data', options=('Location', 'Admission numbers', 'Beds…

### Removing Data Step 1
> We begin by making a copy of the main dataframe

In [65]:
# create a dataframe copy
df_new=df

df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Columns: 157 entries, ('Location', 'Region') to ('Asthma: Process of transitioning young people from paediatric - adult services includes ensuring:', 'No formal transition plan arrangements')
dtypes: float64(25), int64(7), object(125)
memory usage: 195.2+ KB


### Removing Data Step 2
> We then delete any undesired categories from our new dataframe, and view how this affects the shape of the dataframe; the number of columns has dropped dramatically

In [66]:
remove_categories = ["Staffing levels: Number of staff posts in respiratory team (Whole time equivalent (WTE))",
                     "Access to specialist staff and services",
                     "Patient access to onsite palliative care",
                     "Days/times senior decision maker from respiratory team (ST3 or above) undertakes ward round of new asthma/COPD patients on MAU/admissions ward",
                     "Days/times senior decision maker from respiratory team (ST3 or above) undertakes ward round of new asthma/COPD patients on respiratory ward(s)",
                     "Days/times senior decision maker from respiratory team (ST3 or above) undertakes ward round of new asthma/COPD patients on other wards",
                     "Availability of respiratory nurse(s) to review COPD patients",
                     "Availability of respiratory nurse(s) to review asthma patients",
                     "Availability of respiratory physiotherapist(s) to review COPD patients",
                     "Availability of respiratory physiotherapist(s) to review COPD patients",
                     "Integration of care",
                     "Outreach early/supported discharge",
                     "In-reach/supported discharge",
                     "Admissions avoidance",
                     "Oxygen assessment service",
                     "Medicine management service",
                     "Smoking cessation advice",
                     "Chronic disease management",
                     "Nebuliser service",
                     "MDT meeting - staff attendance: Physician posts",
                     "MDT meeting - staff attendance: Nurse posts",
                     "MDT meeting - staff attendance: Physiotherapist posts",
                     "MDT meeting - staff attendance: Other healthcare professional posts",
                     "Asthma: Process of transitioning young people from paediatric - adult services includes ensuring:"]

# ensure remove_categories exist in the dataframe:
to_delete = [x for x in remove_categories if x in list_level_zero_headers(df_new)]

# removal of entire categories of data
df_new=df_new.drop(columns=remove_categories,axis=1,level=0)

df_new.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Columns: 48 entries, ('Location', 'Region') to ('Patient/carer engagement', 'Patient representation within strategic group')
dtypes: float64(1), int64(7), object(40)
memory usage: 59.8+ KB


### Removing Data Step 3
> We then delete any undesired subcategories from our new dataframe, and view how this affects the shape of the dataframe; the number of columns has dropped dramatically

In [67]:
remove_sub_categories = ["Hospital name (d = 159)",
                         "Integrated Care System (ICS)",
                         "Trust/Local Health Board name",
                         "Hospital name (d = 159)",
                         "Number of respiratory coded emergency admissions in 2020/21 financial year per respiratory bed",
                         "Number of COPD coded emergency admissions in 2020/21 financial year per 1,000 adult medical emergency admissions ",
                         "Number of asthma coded emergency admissions in 2020/21 financial year per 1,000 adult medical emergency admissions ",
                         "National average proportion of asthma coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year ",
                         "Number of Level 2 beds available on dedicated respiratory ward(s)",
                         "Number of Level 2 beds available on dedicated respiratory ward(s) per 1,000 adult respiratory admissions",
                         "Hospital has High Dependency Unit(s) (HDUs) to which respiratory patients can be admitted",
                         "Number of HDU beds available to which COPD and asthma patients can be admitted",
                         "Number of HDU beds available to which COPD and asthma patients can be admitted per 10,000 adult medical emergency admissions",
                         "Hospital has Respiratory Support Unit (RSU) to which respiratory patients can be admitted",
                         "Number of RSU beds available to which COPD and asthma patients can be admitted",
                         "Number of RSU beds available to which COPD and asthma patients can be admitted per 10,000 adult medical emergency admissions ",
                         "Name of EPR supplier (Free text)"]
# check subcategories haven't already been removed:
to_delete = [x for x in remove_sub_categories if x in list_level_one_headers(df_new)]

# removal of subcat columns
df_new=df_new.drop(columns=to_delete,axis=1,level=1)

# view the new shape of the data
df_new.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Columns: 35 entries, ('Location', 'Region') to ('Patient/carer engagement', 'Patient representation within strategic group')
dtypes: float64(1), int64(5), object(29)
memory usage: 43.6+ KB


### View Data By Category Post Dropping

In [68]:
show_data_by_category(df_new)

,Region
0,East of England
1,East of England


interactive(children=(Dropdown(description='Category of Data', options=('Location', 'Admission numbers', 'Beds…

In [69]:
df0=len(list_level_zero_headers(df))
df1=len(list_level_one_headers(df))
df_new0=len(list_level_zero_headers(df_new))
df_new1=len(list_level_one_headers(df_new))

print("Number of zero level headers before drop: {}\n".format(df0))
print("Number of first level headers before drop: {}\n".format(df1))

print("Number of zero level headers after drop: {}\n".format(df_new0))
print("Number of first level headers after drop: {}\n".format(df_new1))

Number of zero level headers before drop: 32

Number of first level headers before drop: 157

Number of zero level headers after drop: 9

Number of first level headers after drop: 35



<br><br><br><br><br><br><br><br>
## Data Cleaning | Further Dataframe Manipulation (In-Situ)
> Now we have a new reduced dataframe, we can proceed to set up in-situ cleaning; we define functions to clean specific types of columns - this is so rows of the dataframe can be taken based on one columns values only, on the fly, and without removing rows of data that could be useful further in the analysis. A categories dictionary is first created, mapping every subcategory to their category (for easy access to multiindex zero level headers).


In [70]:
cd=convert_column_indexes_to_dict(df_new)

print(find_category(cd, "Region"))

Location


### Datatypes
> The shape of our new dataframe, df_new, has datatypes `int64(5), object(31)`. 
<br><br>
We begin by analysing which columns contain object types and why.
<br><br>
We then demonstrate how the column may be cleaned on-demand when needed

In [71]:
# show data with object types
show_data_by_category(df_new.select_dtypes(object), len(df_new.index))

,Region
0,East of England
1,East of England
2,East of England
3,East of England
4,East of England
5,East of England
6,East of England
7,East of England
8,East of England
9,East of England


interactive(children=(Dropdown(description='Category of Data', options=('Location', 'Admission numbers', 'Beds…

<br>

### Mixed Numeric and String - Take Numeric Rows Only

#### Function definitions:


In [72]:
def check_non_numeric(df, cat, column_name):
    # this function takes in a dataframe, category, and column name
    
    # note: apply() and isnumeric() combined result in halving compute time compared to to_numeric()    
    if df[cat][column_name].dtype=='int64':
        bools=[[True]]*len(df[cat][column_name].index)
    else:
        bools=np.transpose([df[cat][column_name].apply(lambda x: x.isnumeric())])
        
    return bools

def are_all_numeric(df, cat, column_name):
    bools = check_non_numeric(df, cat, column_name)
    
    return (0 not in bools)

def return_numeric_only_dataframe_column(df, cat, column_name):
    # returns the dataframe column with numeric values only
    bools=check_non_numeric(df, cat, column_name)
    
    df_temp=df_new[cat].iloc[bools]
    
    data=df_temp[[column_name]]

    return data

def return_numeric_only_dataframe(df, cat, column_name):
    # returns the entire dataframe having removed non-numerics from a specified column
    bools=check_non_numeric(df, cat, column_name)
    
    data=df_new.iloc[bools]

    return data


<br>

### Numeric Cleanup Example

> Some of the data columns are set as `object` type because of values being both numeric and string.
<br><br>
> Here we verify the function definitions above work and remove non-numeric rows  


In [73]:
# setup column of data with numeric and string values
category="Admission numbers"
subcategory="Total number of COPD coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year"

# show column of data before cleaning
show_specific_columns_of_data(df_new, [subcategory],100)

,Admission numbers
,"Total number of COPD coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year"
0,316
1,123
2,218
3,116
4,293
5,275
6,46
7,196
8,0


interactive(children=(IntSlider(value=100, description='rows', max=159, min=1), Output()), _dom_classes=('widg…

> We see, upon scrolling through the column above, that the column contains both numeric and string values

In [74]:
# cleanup: return the numeric only column
column_numeric=return_numeric_only_dataframe_column(df_new, category, subcategory)

# display column_numeric, the numeric only column
show_data_with_row_control(column_numeric, len(column_numeric.index))

,"Total number of COPD coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year"
0,316
1,123
2,218
3,116
4,293
5,275
6,46
7,196
8,0
9,43


interactive(children=(IntSlider(value=146, description='rows', max=146, min=1), Output()), _dom_classes=('widg…

> We see above that now a dataframe with the numeric rows of the column are returned - 146 rows, 13 less than the original 159. Executing the function `return_numeric_only_dataframe(df_new, category, subcategory)` will return the entire dataframe, `df_new` with only the rows in which the column "Total number of COPD coded emergency admissions who were discharged from, or died on, dedicated respiratory ward(s) in the 2020/21 financial year" contains numeric values. These functions may be applied the same way to any column of data. 

<br>

### Nominal Yes/No Data - Removing *Other*
> In the same manner as we defined functions to clean numeric columns, we define functions to clean yes-no columns, to be used on the fly.
#### Function definitions:


In [75]:
def check_yes_no(df, cat, column_name):
    # this function takes in a dataframe, category, and column name
    
    # note: apply() and isnumeric() combined result in halving compute time compared to to_numeric()    
    bools=np.transpose([df[cat][column_name].apply(lambda x: x in ["Yes","No"])])
    
    return bools

def return_yesno_only_dataframe_column(df, cat, column_name):
    # returns the dataframe column with numeric values only
    bools=check_yes_no(df, cat, column_name)
    
    df_temp=df_new[cat].iloc[bools]
    
    data=df_temp[[column_name]]

    return data

def return_yesno_only_dataframe(df, cat, column_name):
    # returns the entire dataframe having removed non-numerics from a specified column
    bools=check_yes_no(df, cat, column_name)
    
    data=df_new.iloc[bools]

    return data
    

> Some of the data columns contain Yes/No data with anomolous other values
<br><br>
> Here we illustrate how we deal with such data.

In [76]:
category="Patient/carer engagement"
subcategory="Trust/Health Board has strategic group for respiratory services"

print("Unique values in column: \n")
for val in df_new[category][subcategory].unique().tolist():
    print(val)

Unique values in column: 

Yes
Not known
No


> We see that the unique values of the columns are `'Yes'`, `'No'` and `'Not known'`. We aim to remove the `'Not known'` rows. 

In [77]:
# return a dataframe column, removing any non yes-no data
column_yesno=return_yesno_only_dataframe_column(df_new, category, subcategory)

print("Unique values in column:\n")
for val in column_yesno[subcategory].unique().tolist():
    print(val)
    
show_data_with_row_control(column_yesno, len(column_yesno.index))

,Trust/Health Board has strategic group for respiratory services
0,Yes
1,Yes
2,Yes
3,Yes
4,Yes
5,Yes
6,Yes
8,Yes
9,Yes
12,Yes


interactive(children=(IntSlider(value=138, description='rows', max=138, min=1), Output()), _dom_classes=('widg…

> We see above that now a dataframe with the numeric rows of the column are returned - 138 rows, 21 less than the original 159. Executing the function `return_yesno_only_dataframe(df_new, category, subcategory)` will return the entire dataframe, `df_new` with only the 138 rows in which the column "Trust/Health Board has strategic group for respiratory services" contains yes-no values. These functions may be applied the same way to any column of data. 

<br>

### Stripping Strings 



In [78]:
# strip all string values in every column of the dataframe df_new
df_obj = df_new.select_dtypes(['object'])
df_new[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

<br><br><br><br><br><br><br><br>
# Mapping Data 

## Questions About The Data
<br>

*To begin our analysis, we ask some ground-level questions about the data.*

<br>
<li> Which regions of the country have the highest admissions to hospital due to respiratory illnesses? </li>
<li> Does the availibility of on-call respiratory consultants have an effect on hospital admissions? </li>
<li> What is the distribution of hospital admissions to beds available? </li>


<br><br>
### Regional Hospital Admissions
<br>

> *Here we begin to answer the question*
<br>
Which regions of the country have the highest admissions to hospital due to respiratory illnesses? 
<br><br>
#### Function definitions:

In [79]:
def sum_by_region(df,data_to_sum):
    # returns the sum of values in data_to_sum per region
    
    # get relevant columns
    regions_df=get_specific_column_from_multiindex(df, "Region").droplevel(0,axis=1)
    df2=get_specific_column_from_multiindex(df,data_to_sum).droplevel(0,axis=1)
    # dataframes to combine
    temp=[regions_df, df2]
    # combine the two dataframes, combined series obtained
    combined_series=pd.concat(temp, axis=1)
    # convert to dataframe, group by region and sum numeric column
    summed_df=combined_series.groupby("Region", as_index=False)[data_to_sum].sum()
    
    return summed_df
    

def tooltip_marker(map_name, locs_dict):
    # returns a tooltip-marker to folium map with region labels 
    
    # get locations and add a tooltip at their specified longitudes and latitudes 
    for key, value in locs_dict.items():
        folium.Marker(value,
                      popup='<i>{}</i>'.format(key),
                      tooltip="{}".format(key)).add_to(map_name)
        
        
def tooltip_marker(map_name, locs_dict, vals_dict):
    # returns a tooltip-marker to folium map with region labels and relevant values
    
    # get locations and add a tooltip at their specified longitudes and latitudes 
    # and add values from val_dict to tooltip
    for key, value in locs_dict.items():
        folium.Marker(value,
                      popup='<i>{}</i>'.format(key),
                      tooltip="{}<br>Admissions: {}".format(key,vals_dict[key])).add_to(map_name)


def display_data_on_map(df_dictionary, text):
    # returns an interactive folium map display
    # with a dropdown for each key in df_dictionary to display different maps
    
    @interact
    def display_map(option=widgets.Dropdown(options=list(df_dictionary.keys()),
                                            description="Data to view",
                                            style={'description_width': '40%'})):
        # create base map
        england=folium.Map(location=[52.8781, -0.5276],zoom_start=5.5)
        
        # get relevant data from dict
        df=df_dictionary[option]
        a=df.columns.tolist()[0]
        b=df.columns.tolist()[1]
        
        # overlay with chloropleth, using boundary data regions_geo 
        folium.Choropleth(
            geo_data = regions_geo,                  #json
            name ='choropleth',                  
            data = df,                     
            columns = [a, b], #columns to work on
            key_on ='feature.properties.nhser19nm',
            fill_color ='YlGnBu',     #I passed colors Yellow,Green,Blue
            fill_opacity = 0.7,
            line_opacity = 0.2,
            legend_name = "Hospital Admissions").add_to(england)
        
        # create a dictionary with admission numbers for tooltip
        admissions=dict(zip(df[a],round(df[b])))
        
        # add region tooltips
        tooltip_marker(england, locs, admissions)
        
        # add title
        title_html = '''<h3 align="center" style="font-size:16px"><b>{} {}</b></h3>'''.format(option, text)
       
        england.get_root().html.add_child(folium.Element(title_html))
        
        # display folium map
        return england

#### Setting up relevant variables:

In [80]:
# set up header values for columns to analyse
data_to_analyse=["Region",
                 "Number of respiratory coded medical emergency admissions in 2020/21 financial year ",
                 "Number of COPD coded emergency admissions in 2020/21 financial year ",
                 "Number of asthma coded emergency admissions  in the 2020/21 financial year ",
                 "Number of beds available on dedicated respiratory ward(s) "]

# get unique region vals
regions = set([s.strip() for s in df_new["Location"]["Region"]])

# group by regions, sum respiratory admissions and drop Wales
regional_resp_admissions=(sum_by_region(df_new, data_to_analyse[1])).drop(labels=7,axis=0)

# group by regions, sum copd admissions and drop Wales
regional_copd_admissions=(sum_by_region(df_new, data_to_analyse[2])).drop(labels=7,axis=0)

# group by regions, sum asthma admission and drop Wales
regional_asthma_admissions=(sum_by_region(df_new, data_to_analyse[3])).drop(labels=7,axis=0)

regional_dict={"Respiratory":regional_resp_admissions, "COPD":regional_copd_admissions, "Asthma":regional_asthma_admissions}

# load .geojson region boundary data
with open('NHS_England_Regions_Map.geojson', 'r') as jsonFile:
    regions_geo = json.load(jsonFile)
    
# dictionary of region longs and lats for tooltip
locs={"East of England":[52.1911, 0.1927], 
      "London":[51.5072, -0.1276], 
      "Midlands":[52.3449, -1.9718],
      "North East and Yorkshire":[54.2333, -1.3364],
      "North West":[53.6221, -2.5945],
      "South East":[51.1781, -0.5596],
      "South West":[50.7772, -3.9995],
      # "Wales":[52.1307, -3.7837]
     }


#### Display the map:

In [81]:
# show data for each group: Respiratory, COPD and Asthma 
display_data_on_map(regional_dict, "Admissions")

interactive(children=(Dropdown(description='Data to view', options=('Respiratory', 'COPD', 'Asthma'), style=De…

#### ANALYSIS OF THE ABOVE MAP
> The above maps show that respiratory illnesses result in the greatest *total number* of respiratory hospital admissions in the North East and Yorkshire region with 91,865 admissions, far exceeding anywhere else. The lowest *total number* of respiratory hospital admissions is in the East of England with 41,609 and the South West closely following with 42,751 admissions; these are likely the cleanest (least polluted) regions in the nation, however services provided may impact this also - further analysis required. London, despire being geographically far smaller, has more admission than both the East of England and the South West, with 53,859 admissions due to respiratory illnesses.
<br><br>
COPD specific admissions see a relative increase in the North West, Midlands and the South West, while London and the South East see a relative decrease. Further analysis is required to understand why. 
<br><br>
Asthma cases see a huge relative increase in the North West, and increases in the Midlands, East of England and London while the North East and Yorkshire sees a massive relative decrease. Further analysis is required to determine whether this is likely due to services provided or pollution (although without pollution data conclusions regarding this cannot be drawn).
<br><br>
> *However, these figures are population dependent. We thus analyse repiratory admissions per bed available in order to make more meaningful analysis, since population data is not available.*


<br><br>
### Regional Hospital Admissions Per Bed
<br>

> *Here we begin to answer the question*
<br>
Which regions of the country have the highest admissions to hospital due to respiratory illnesses? 
<br><br>
> *The dataset contains two relevant columns of data, "Number of beds available on dedicated respiratory ward(s) " and "Number of available medical beds", however, we whether the columns are entirely numeric.*

#### Check respiratory beds available and total beds available are all numeric:

In [82]:
category="Beds for admitted patients"
column="Number of beds available on dedicated respiratory ward(s) "

# get available respiratory wards beds dataframe column
resp_beds=get_specific_column_from_multiindex(df_new,column)

# check if data is numeric only and print statement
mess='\nIs "Number of beds available on dedicated respiratory ward(s)" entirely numeric: {} '\
    .format(are_all_numeric(resp_beds,category,column))
print("\n{}".format(mess))

# get total available beds dataframe column
column="Number of available medical beds"
beds=get_specific_column_from_multiindex(df_new, column)

# check if data is numeric only and print statement
mess='Is "Number of available medical beds" entirely numeric: {}'\
    .format(are_all_numeric(beds, category, column))
print("\n{}".format(mess))




Is "Number of beds available on dedicated respiratory ward(s)" entirely numeric: False 

Is "Number of available medical beds" entirely numeric: True


> We see that "Number of beds available on dedicated respiratory ward(s)" contains non-numeric data; this is because not every hospital has a dedicated respiratory ward. In order to include as many hospitals hospital in our analysis, we look at the admissions relative to the total number of bed available.
<br><br>
> *Thus, we being to look at admission data per bed availible to assess service overwhelm, looking at admissions with respect to all beds available*


#### View the relevant columns of data: 

In [83]:
# list of headers for relevant columns
data_to_analyse=["Region",
                 "Number of adult medical emergency admissions in 2020/21 financial year ",
                 "Number of respiratory coded medical emergency admissions in 2020/21 financial year ",
                 "Number of COPD coded emergency admissions in 2020/21 financial year ",
                 "Number of asthma coded emergency admissions  in the 2020/21 financial year ",
                 "Number of available medical beds"]

# view data in columns 
show_specific_columns_of_data(df_new, data_to_analyse)


Location  \
            Region   
0  East of England   
1  East of England   

                                                        Admission numbers  \
  Number of adult medical emergency admissions in 2020/21 financial year    
0                                              39628                        
1                                              13639                        

                                                                                       \
  Number of respiratory coded medical emergency admissions in 2020/21 financial year    
0                                               5265                                    
1                                               2482                                    

                                                                        \
  Number of COPD coded emergency admissions in 2020/21 financial year    
0                                                806                     
1                                                346                     

                                                                               \
  Number of asthma coded emergency admissions  in the 2020/21 financial year    
0                                                369                            
1                                                109                            

        Beds for admitted patients  
  Number of available medical beds  
0                              380  
1                              319

interactive(children=(IntSlider(value=2, description='rows', max=159, min=1), Output()), _dom_classes=('widget…

#### Data-Cleaning | Dropping Column With 0 Value

> We see that there exists at least one 0 value in "Number of available medical beds"; such rows are omitted for this part of the analysis (where column dividing will take place), as follows:

In [84]:
# new function definition to sum by region for non-multiiindex datafram 
def sum_by_region2(df_region,new_col,data_to_sum):
    # returns the sum of values in new_col[data_to_sum] per region
    
    # get column to sum values 
    df1=new_col[data_to_sum]
    
    # dataframes to combine
    temp=[df_region, df1]
    
    # combine the two dataframes, combined series obtained
    combined_series=pd.concat(temp, axis=1)
    # convert to dataframe, group by region and sum numeric column
    summed_df=combined_series.groupby("Region", as_index=False)[data_to_sum].sum()
    
    return summed_df

# get number of beds column 
number_of_beds=get_specific_column_from_multiindex(df_new, data_to_analyse[5]).droplevel(0,axis=1)

# create a list of booleans to map non-zero values to 1 and 0 values to 0
bools=np.transpose([number_of_beds[data_to_analyse[5]].apply(lambda x: x!=0)])

# create a dataframe with non-zero bed value rows only
df_new_temp=df_new.iloc[bools]

# define new single column regions dataframe
regions_df=get_specific_column_from_multiindex(df_new_temp, "Region").droplevel(0,axis=1)


# Compute respiratory admissions per bed available
name=data_to_analyse[2]
resp_admissions=get_specific_column_from_multiindex(df_new_temp, name).droplevel(0,axis=1)

x=resp_admissions[name]/number_of_beds[data_to_analyse[5]]
df_resp_per_bed=pd.DataFrame(x,columns=["Respiratory Admissions Per Bed Available"])

# Compute copd admissions per bed available
name=data_to_analyse[3]
copd_admissions=get_specific_column_from_multiindex(df_new_temp, name).droplevel(0,axis=1)

x=copd_admissions[name]/number_of_beds[data_to_analyse[5]]
df_copd_per_bed=pd.DataFrame(x,columns=["COPD Admissions Per Bed Available"])

# Compute asthma admissions per bed available
name=data_to_analyse[4]
asthma_admissions=get_specific_column_from_multiindex(df_new_temp, name).droplevel(0,axis=1)

x=asthma_admissions[name]/number_of_beds[data_to_analyse[5]]
df_asthma_per_bed=pd.DataFrame(x,columns=["Asthma Admissions Per Bed Available"])

# sum respiratory admissions per bed by region
reg_resp_per_bed=sum_by_region2(regions_df,df_resp_per_bed,"Respiratory Admissions Per Bed Available")
# sum copd admissions per bed by region
reg_copd_per_bed=sum_by_region2(regions_df,df_copd_per_bed,"COPD Admissions Per Bed Available")
# sum asthma admissions per bed by region
reg_asthma_per_bed=sum_by_region2(regions_df,df_asthma_per_bed,"Asthma Admissions Per Bed Available")

# create dictionary of dataframes to view data on folium map
regional_dict_per_bed={"Respiratory":reg_resp_per_bed, "COPD":reg_copd_per_bed, "Asthma":reg_asthma_per_bed}

In [85]:
# display folium map 
display_data_on_map(regional_dict_per_bed, "Admissions Per Bed")

interactive(children=(Dropdown(description='Data to view', options=('Respiratory', 'COPD', 'Asthma'), style=De…

#### ANALYSIS OF THE ABOVE MAP 
> Analysing now per bed, we see that the North East and Yorkshire has the highest admissions relative to the number of beds available, indicating the greatest strain on the NHS services in that region due to respiratory illnesses. Per bed, admissions. Both the South West and London have gone up in their values relative to the rest of the country, indicating greater strain on their NHS services than elsewhere. 
<br><br>
> Of respiratory illnesses, the South West experiences a large portion of its strain on service due to COPD admissions as seen by its COPD admissions per bed available being the highest in the country, with 74 admissions per bed (the North East and Yorkshire has the highest with 87). 
<br><br>
> Of respiratory illnesses, a concentration of strain on service due to asthma appears to be inthe North West while the North East and Yorkshire shows a drop in strain due to asthma relative to all respiratory illnesses and COPD. The South West experience less strain on service due to asthma, as does the South East. 

#### Bar chart of admissions per bed

In [86]:
# list of header titles 
headers=["Respiratory Admissions Per Bed Available",\
    "COPD Admissions Per Bed Available",\
    "Asthma Admissions Per Bed Available"]

# create list of dataframe columns to concatenate
temp=[regions_df, df_resp_per_bed, df_copd_per_bed, df_asthma_per_bed]
# concat returns columns joined side by side, as a series
combined_series=pd.concat(temp, axis=1)
# group series by region and sum values in each column by region
summed_df=combined_series.groupby("Region", as_index=False)\
    [headers].sum()

# set up bar chart figure
fig = px.bar(summed_df, x="Region", y=headers, title="Wide-Form Input")

# plot each column in summed_df by region
fig=go.Figure(data=[
    go.Bar(name="Respiratory Admissions",x=summed_df["Region"],y=summed_df[headers[0]]),
    go.Bar(name="COPD Admissions",x=summed_df["Region"],y=summed_df[headers[1]]),
    go.Bar(name="Asthma Admissions",x=summed_df["Region"],y=summed_df[headers[2]]),
])

# add a title
fig=fig.update_layout(title_text="Admissions per Bed")


In [87]:
# display the bar chart
fig.show()

#### ANALYSIS OF ABOVE BAR CHART
> It is clear from the bar chart above that the region with the greatest strain on its service due to resporatory illnesses is the North East and Yorkshire, while the North West of England has a relatively high strain from asthma admissions. 
<br><br>
> The South West of England has a significantly low strain due to asthma admissions, and the East of England has the lowest strain on service overall due to any respiratory illness.  
<br>
The data will be analysed further to make inferrences as to why.


<br><br>
### Analysis of Pulmonary Rehabillitation for COPD Patients
> Having analysed the number of admissions per region, we are now able to assess the effectiveness of different hospital factors, beginning with pulmonary rehabillitation. 

#### Data Wrangling | Map "Yes" to 1 and "No" to 0

In [88]:
# remove 'Not known' rows from relevant column 
df_new_temp=return_yesno_only_dataframe(df_new,\
                                   "Pulmonary rehabilitation (PR) service available to COPD patients discharged following flare-up",\
                                   "Is this/are these services available within 30 days of discharge")

# map "Yes" to 1 and "No" to 0
df_new_temp=df_new_temp.replace({"Yes":1,"No":0})


#### Plot rehab availability per region

In [89]:

# drop level 0 headers and obtain level 1 header names 
# columns=df_new_temp.droplevel(0, axis=1).columns.tolist()

# list of headers for columns of data 
data_to_analyse=["Region", "Is this/are these services available within 30 days of discharge"]

# sum yes and no values by region
df_regions_copd_rehab=sum_by_region(df_new_temp, data_to_analyse[1])

# Set up bar chart figure
fig=go.Figure(data=[
    go.Bar(name="COPD Rehab Availability",
           x=df_regions_copd_rehab["Region"],
           y=df_regions_copd_rehab[data_to_analyse[1]])
])

# add title to figure
fig=fig.update_layout(title_text="COPD Rehab Availability per Region")


In [90]:
# display the bar chart
fig.show()

#### ANALYSIS OF THE BAR CHART ABOVE

<br>

> We see the greatest number of hospitals with rehab available for COPD patients is in London, which is interesting given London has a relatively higher COPD proportion of admissions than all respiratory relative to the rest of the nation. 
<br><br>
> The South East and South West both also have high numbers of hospitals with rehab, likely necessary due to a higher strain on service (as shown on chloropleths for admissions per region) whilst the East of England has the lowest, as well as the lowest COPD strain, thus not necessary for more COPD rehab facilities. 
<br><br> 
> Service in North East and Yorkshire and Midlands is poor given high strains on service due to COPD relative to the rest of the nation. 

### Distribution of the Number of Hospital Admissions

In [91]:
summed_df

,Region,Respiratory Admissions Per Bed Available,COPD Admissions Per Bed Available,Asthma Admissions Per Bed Available
0,East of England,109.582254,30.531986,12.648976
1,London,198.738348,45.262910,13.545870
2,Midlands,187.871678,65.588193,19.259351
3,North East and Yorkshire,313.124697,86.575575,18.884880
4,North West,211.905690,67.024039,31.773033
5,South East,199.023006,45.230275,8.776572
6,South West,156.585284,74.329225,7.027751
7,Wales,99.326970,29.120345,12.516637


In [92]:
name="Number of adult medical emergency admissions in 2020/21 financial year per medical bed"
admissions_per_bed=get_specific_column_from_multiindex(\
                                       df_new, name).droplevel(0,axis=1)

admissions_per_bed=admissions_per_bed.rename(columns={
    name: "Medical Emergency Admissions Per Bed Available"
})

# plot using plotly.express
fig_hist=px.histogram(admissions_per_bed,
                      x="Medical Emergency Admissions Per Bed Available")

# plot using plotly.express
# fig=px.histogram(df_resp_per_bed,x="Respiratory Admissions Per Bed Available")
# fig.show()

In [93]:
fig_hist.show()

> We see that the largest proportion of hospitals sit in the range 50-59 admissions per hospital bed in 2020/21. To investigate whether COVID-19 had an impact on hospital admissions, in a similar fashion, we investigate the distribution of respiratory admissions as a proportion of total admissions for respiratory illnesses in general and compare with COPD and Asthma admissions. 